In [1]:
from mismo.datasets import load_febrl1

import modin.pandas as pd

In [2]:
df, labels = load_febrl1()
df

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
0,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
1,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
2,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
3,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
4,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933
...,...,...,...,...,...,...,...,...,...,...
995,stephanie,geu,28,bainton crescent,masonic memorial village,maryborough,2541,sa,19421008,3997529
996,nicholas,NaN,289,britten-jonues drive,jabaru court,paddington,2000,vic,19970422,5062738
997,lachlan,katsiavos,29,paul coe cdrescent,NaN,casual,2913,nsw,19380406,4112327
998,monique,gergely,21,harwoos court,hyberni a park,sherwood,2207,nsw,19790807,7375144


In [5]:
from mismo.block import Equals, FingerprintBlocker

In [24]:
b = FingerprintBlocker(Equals("surname"))
b

In [27]:
block_map = b.block(df, df)
block_map

,index_1,index_2
0,0,0
1,1,1
2,1,46
3,1,222
4,1,258
...,...,...
4715,998,998
4716,999,127
4717,999,536
4718,999,594


In [ ]:
universe = pd.Series([0, 1, 2, 3, 4])
s0 = pd.Series([0, 1])  # this chosen first
s1 = pd.Series([0])  # shouldn't get chosen
s3 = pd.Series([2, 5])  # this chosen second
s2 = pd.Series([2, 5, 6])  # this not chosen

In [29]:
type(block_map)

modin.pandas.dataframe.DataFrame